# BinaPs Demo

Copyright 2022 Bernardo C. Rodrigues

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public
License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later
version. This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the
implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details. You should have received a copy of the GNU General Public License along with this program. If not, 
see <https://www.gnu.org/licenses/>. 

This notebook demonstrates the usability of the BinaPs implementation provided by its authors. It consists on three
steps:

**1. Generate synthetic data:** With one of the provided scripts, we generate synthetic data in which a set of known
patterns are planted.

**2. Run BinaPs:** We'll run BinaPs over this synthetic dataset and get a list of inferred patterns

**3. Compare inferred to original patterns**: We use another provided script to calculate the 
[F1](https://en.wikipedia.org/wiki/F-score) value between the inferred and original patterns.

In [1]:
import torch

# Check if CUDA is available
if(torch.cuda.is_available()):
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
else:
    print("No CUDA device avaiable")

CUDA Device: NVIDIA GeForce GTX 1050


## 1. Generate synthetic data

In this step we will end up with 4 files:

```
...
├── data.dat
├── data.dat_patterns.txt
├── data_itemOverlap.dat
├── data_itemOverlap.dat_patterns.txt
...
```

*data.dat* and *data_itemOverlap.dat* are the synthetic datasets while *data.dat_patterns.txt* are their respective
*data_itemOverlap.dat_patterns.txt* root patterns respectively. Arbitrary noise is added to test the BinaPS robustness.

In *data_itemOverlap.dat* the patterns may overlap (e.g ABC CDE CEF) while in *data.dat* they may not (e.g. AB C DE F).


In [2]:
from lib.utils import generate_synthetic_data
import os

output_file = f"data"

row_quantity = 10000
column_quantity = 50
max_pattern_size = 10
noise = 0.001
density = 0.05

generate_synthetic_data(row_quantity, column_quantity, output_file, max_pattern_size, noise, density)

Rscript binaps/Data/Synthetic_data/generate_toy.R AND 50 10000 10 data 0.001 0.05
[1] 310
[1] 10000   310
[1] "Added noise."
[1] "Converted to dat file."
[1] "Removed rows without content."
         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 380919 20.4     809590 43.3   641325 34.3
Vcells 892617  6.9    8388608 64.0  8388514 64.0
[1] 10000   186



## 2. Run BinaPs

A complete run of th BinaPs autoencoder. In this step we'll end up with the inferred patterns:
```
...
├── data.binaps.patterns
...
```

Attention, patterns in this output start at '0', as opposed to patterns generated by
*generate_synthetic_data* that start at '1'. In other words, the pattern {0,1,5} is equal to {1,2,6}
in the original dataset.


In [3]:
from lib.utils import run_binaps

data_path = "data.dat"
hidden_dimension = 20
epochs = 100

run_binaps(data_path, hidden_dimension, epochs)

python3 binaps/Binaps_code/main.py -i data.dat --hidden_dim 20 --epochs=100
Data Sparsity:
0.05372346044153375
Train Epoch: 1 [0/8521 (0%)]	Loss: 14.032347
Train Epoch: 1 [640/8521 (7%)]	Loss: 16.360149
Train Epoch: 1 [1280/8521 (15%)]	Loss: 12.570875
Train Epoch: 1 [1920/8521 (22%)]	Loss: 12.740749
Train Epoch: 1 [2560/8521 (30%)]	Loss: 11.248817
Train Epoch: 1 [3200/8521 (37%)]	Loss: 11.507511
Train Epoch: 1 [3840/8521 (45%)]	Loss: 10.967735
Train Epoch: 1 [4480/8521 (52%)]	Loss: 8.721739
Train Epoch: 1 [5120/8521 (60%)]	Loss: 9.918232
Train Epoch: 1 [5760/8521 (67%)]	Loss: 7.207335
Train Epoch: 1 [6400/8521 (75%)]	Loss: 8.313211
Train Epoch: 1 [7040/8521 (82%)]	Loss: 7.175420
Train Epoch: 1 [7680/8521 (90%)]	Loss: 6.401906
Train Epoch: 1 [8320/8521 (97%)]	Loss: 5.730946

Test set: Average loss: 85.639381, Accuracy: 0/946 (0%)

Train Epoch: 2 [0/8521 (0%)]	Loss: 5.441792
Train Epoch: 2 [640/8521 (7%)]	Loss: 5.075443
Train Epoch: 2 [1280/8521 (15%)]	Loss: 5.515021
Train Epoch: 2 [1920

## 3. Compare inferred to original patterns

Get the F1 score for the inferred patterns.

compare_datasets_based_on_f1 takes into account the diference of the patterns' start mentioned at
the previous step.

In [4]:
from lib.utils import compare_datasets_based_on_f1

estimated_patterns_file = "data.binaps.patterns"
real_patterns_file = "data.dat_patterns.txt"

compare_datasets_based_on_f1(estimated_patterns_file, real_patterns_file)

0.1111111111111111



# Overlapped patterns example

In [5]:
data_path = "data_itemOverlap.dat"
hidden_dimension = 20
epochs = 100

run_binaps(data_path, hidden_dimension, epochs)

python3 binaps/Binaps_code/main.py -i data_itemOverlap.dat --hidden_dim 20 --epochs=100
Data Sparsity:
0.0821346239322735
Train Epoch: 1 [0/8405 (0%)]	Loss: 13.846106
Train Epoch: 1 [640/8405 (8%)]	Loss: 12.155611
Train Epoch: 1 [1280/8405 (15%)]	Loss: 9.976454
Train Epoch: 1 [1920/8405 (23%)]	Loss: 10.460820
Train Epoch: 1 [2560/8405 (30%)]	Loss: 9.764177
Train Epoch: 1 [3200/8405 (38%)]	Loss: 9.066208
Train Epoch: 1 [3840/8405 (45%)]	Loss: 8.574670
Train Epoch: 1 [4480/8405 (53%)]	Loss: 8.297632
Train Epoch: 1 [5120/8405 (61%)]	Loss: 8.202707
Train Epoch: 1 [5760/8405 (68%)]	Loss: 7.360813
Train Epoch: 1 [6400/8405 (76%)]	Loss: 7.047603
Train Epoch: 1 [7040/8405 (83%)]	Loss: 6.951677
Train Epoch: 1 [7680/8405 (91%)]	Loss: 5.975879
Train Epoch: 1 [8320/8405 (98%)]	Loss: 6.547225

Test set: Average loss: 92.769867, Accuracy: 0/933 (0%)

Train Epoch: 2 [0/8405 (0%)]	Loss: 5.904584
Train Epoch: 2 [640/8405 (8%)]	Loss: 5.254344
Train Epoch: 2 [1280/8405 (15%)]	Loss: 5.673965
Train Epoch: 

In [6]:
estimated_patterns_file = "data_itemOverlap.binaps.patterns"
real_patterns_file = "data_itemOverlap.dat_patterns.txt"

compare_datasets_based_on_f1(estimated_patterns_file, real_patterns_file)

0.1917808219178082

